# 初始化

In [1]:
import os
import sys
from pathlib import Path
import time
import json

# 添加项目根目录到Python路径
sys.path.insert(0, str(Path.cwd()))

# 从最新的ReactAgentMinimal导入
from core.react_agent_minimal import ReactAgentMinimal



work_dir = "/home/guci/robot_projects/blog/"

  ✅ 已加载11个环境变量: /home/guci/aiProjects/mda/pim-compiler/.env


# 清空目录

In [ ]:
# 创建代码生成Agent
agent = ReactAgentMinimal(
    work_dir='/home/guci/robot_projects',
    name="generation_agent",
    description="专家",
    model="deepseek-chat",  # x-ai/grok-code-fast-1

)



# 压缩blog目录

In [ ]:
task="删除blog.zip .  把blog目录压缩成一个zip文件"
x=agent.execute(task=task)
print(x)

# 删除blog目录

In [ ]:
task="删除blog目录"
x=agent.execute(task=task)
print(x)

In [ ]:
task="创建blog目录"
x=agent.execute(task=task)
print(x)

In [ ]:
task="解压blog.zip到blog目录"
x=agent.execute(task=task)
print(x)

In [ ]:
task="清空blog目录" # 只保留blog_psm.md
x=agent.execute(task=task)
print(x)

# 生成psm

In [ ]:
pim_file="/home/guci/aiProjects/mda/pim-compiler/examples/blog.md"

# 创建PSM生成Agent（改回DeepSeek - 更稳定）
psm_generation_agent = ReactAgentMinimal(
    work_dir=work_dir,
    name="psm_generation_agent",
    description="PSM生成专家 - 根据PIM生成平台特定模型",
    model="kimi-k2-turbo-preview",  # x-ai/grok-code-fast-1 使用DeepSeek模型（经过验证，稳定高效）
    knowledge_files=[
        "knowledge/mda/pim_to_psm_knowledge.md",  # PSM结构定义
        "knowledge/large_file_handling.md"        # 通用文件处理策略
        # structured_notes.md 自动加载
    ]
)

# 执行PSM生成任务
import time
start_time = time.time()

x = psm_generation_agent.execute(task=f"""
# PIM文件                                
{pim_file}

# 任务
1: 删除blog_psm.md
2：根据上面的PIM生成完整的PSM文档，文件名是blog_psm.md

# 成功判定条件
1. blog_psm.md文件存在
2. 包含Domain Models章节
3. 包含Service Layer章节  
4. 包含REST API Design章节
5. 包含Application Configuration章节
6. 包含Testing Specifications章节


""")

# 报告执行时间
elapsed = time.time() - start_time
print(f"\n⏱️ PSM生成耗时: {elapsed:.2f}秒")

if x is None:
    print("返回结果为空")
else:
    print(f"返回结果：{x[:500]}...")  # 只显示前500个字符

# large_file_handling.md知识文件优化

In [ ]:
import sys
import os
from pathlib import Path
sys.path.append('core')
sys.path.append('core/tools')

from core.react_agent_minimal import ReactAgentMinimal
from core.tools.create_agent_tool import CreateAgentTool

print("🧠 启动元认知Agent优化PSM生成流程")
print("="*60)

# 1. 创建元认知Agent
meta_cognitive_agent = ReactAgentMinimal(
    work_dir=".",
    name="meta_cognitive_agent",
    description="元认知专家 - 通过分析和改进知识文件来优化Agent性能",
    model="deepseek-chat",  # 使用DeepSeek模型
    knowledge_files=[
        "knowledge/reinforcement_learning_optimization.md"
    ]
)

# 2. 添加create_agent工具
create_tool = CreateAgentTool(work_dir=".", parent_agent=meta_cognitive_agent)
meta_cognitive_agent.append_tool(create_tool)

# 3. 执行元认知优化任务
optimization_task = """
# 元认知任务：通过优化通用知识来提升PSM生成效率

## 问题分析
PSM生成Agent使用了16轮才完成任务，目标是10轮以内。
核心问题：Agent不知道如何高效地一次性生成结构化文档。

## 你的任务
1. **分析**当前的`knowledge/large_file_handling.md`
2. **优化**其中的通用策略，使其更清晰、更有指导性
3. **创建测试Agent**验证PSM生成效率
4. 如果仍超过10轮，继续优化知识文件

## 优化方向
在`large_file_handling.md`中强化以下通用原则：
- 更明确的"一次性生成"示例
- 更清晰的Python脚本模板
- 强调"先构建完整内容，后写入"
- 提供更具体的代码模板

## ⚠️ 限制
- **只修改**`knowledge/large_file_handling.md`
- **保持通用性**，不要添加PSM特定内容
- **不创建**新的知识文件
- **不创建**.py可执行文件

## 测试配置
```python
# 创建PSM测试Agent
psm_agent = ReactAgentMinimal(
    work_dir="output/blog",
    name="psm_test_agent",
    model="deepseek-chat",
    knowledge_files=[
        "knowledge/mda/pim_to_psm_knowledge.md",  # PSM结构定义
        "knowledge/large_file_handling.md"        # 你优化的通用知识
    ]
)

# 测试任务
task = '''
根据PIM文件 /home/guci/aiProjects/mda/pim-compiler/examples/blog.md 
生成完整的PSM文档 blog_psm_optimized.md

要求：
1. 必须在10轮以内完成
2. 使用一次性生成策略
'''
```

## 成功标准
- PSM生成在**10轮以内**完成
- 只修改了`large_file_handling.md`
- 知识文件保持**通用性**
- 没有创建新文件

## 提示
好的通用知识应该：
- 提供清晰的决策树
- 给出具体的代码模板
- 强调最佳实践
- 警示常见错误

记住：通过改进**通用知识**来帮助**所有**文档生成任务，而不仅仅是PSM。
"""

# 执行元认知优化
result = meta_cognitive_agent.execute(task=optimization_task)

print("\n" + "="*60)
print("✅ 元认知优化完成")
print(f"结果摘要: {result[:500]}..." if result else "无结果")
print("="*60)

# 根据psm生成代码

In [ ]:
# 创建代码生成Agent
generation_agent = ReactAgentMinimal(
    work_dir=work_dir,
    name="generation_agent",
    description="代码生成专家 - 根据PSM生成FastAPI代码",
    model="kimi-k2-turbo-preview",  # 使用DeepSeek模型
    knowledge_files=[
        "knowledge/mda/generation_knowledge.md"  # 只需要生成知识
        # structured_notes.md 自动加载，包含验证
    ],
    # api_key= os.getenv("OPENROUTER_API_KEY"),
    # base_url="https://openrouter.ai/api/v1"
)

# 执行代码生成任务
x = generation_agent.execute(task="""
# psm文件                                
blog_psm.md

# 任务
根据上面的psm生成代码

# 成功判定条件
1. app/main.py文件存在
2. app/models.py文件存在
3. app/schemas.py文件存在
4. tests/目录存在且包含测试文件
5. readme.md文件存在

# 验证要求【强制执行】
完成生成后必须：
1. 使用list_directory检查app/目录
2. 使用list_directory检查tests/目录  
3. 使用read_file验证readme.md存在
4. 如果任何文件缺失，继续生成或明确报告失败原因
""")

if x is None:
    print("返回结果为空")
else:
    print(f"返回结果：{x[:500]}...")  # 只显示前500个字符

## 复制buggy code到当前目录

In [ ]:
agent = ReactAgentMinimal(
    work_dir=work_dir,
    name="debug_agent",
    description="调试修复专家 - 修复代码和测试问题",
    model="deepseek-chat", 
)

x=agent.execute(task="""
# 任务
1：清空当前工作目录
2：复制buggy code到当前目录

# buggy code目录
/home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need/output/buggy_code

""")

print(x)

# 修复单元测试

In [ ]:
# 创建调试修复Agent
debug_agent = ReactAgentMinimal(
    work_dir=work_dir,
    name="debug_agent",
    description="调试修复专家 - 修复代码和测试问题",
    model="kimi-k2-turbo-preview",  # 使用DeepSeek模型
    knowledge_files=[
        "knowledge/mda/debugging_unified.md"  # 使用统一的调试知识文件
    ],
    # api_key= os.getenv("OPENROUTER_API_KEY"),
    # base_url="https://openrouter.ai/api/v1",
    max_rounds=300
)

# 执行调试修复任务
x = debug_agent.execute(task="""
# 任务
修复单元测试，让所有测试通过

# 输出
单元测试成功率

# 成功判定条件
单元测试百分之百通过

# 验证要求【强制】
完成修复后必须：
1. 运行单元测试
2. 验证输出中没有失败的测试
3. 如果有失败，继续修复
""")

if x is None:
    print("返回结果为空")
else:
    print(f"返回结果：{x[:500]}...")  # 只显示前500个字符

## 运行单元测试

In [ ]:

# 执行调试修复任务
x = debug_agent.execute(task="""
# 任务
运行单元测试

使用execution context工具
""")

if x is None:
    print("返回结果为空")
else:
    print(f"返回结果：{x[:500]}...")  # 只显示前500个字符

In [ ]:

# 执行调试修复任务
x = debug_agent.execute(task="""
这个软件的单元测试框架是unittest还是pytest？
""")

print(x)

# 运行app

In [ ]:
# 创建运行应用Agent
run_app_agent = ReactAgentMinimal(
    work_dir=work_dir,
    name="run_app_agent",
    description="应用运行专家 - 启动和管理FastAPI应用",
    model="deepseek-chat"  # 使用DeepSeek模型
)

# 执行运行应用任务
x = run_app_agent.execute(task="""
# 任务
1：如果app已经运行，停止app
2：运行app

# 知识
使用curl命令必须加上--noproxy参数
""")

if x is None:
    print("返回结果为空")
else:
    print(f"返回结果：{x[:500]}...")  # 只显示前500个字符

In [ ]:
x = run_app_agent.execute(task="""
停止App
""")

print(x)

# mda工作流

In [ ]:
print("🚀 MDA工作流演示 - Agent作为Function")
print("=" * 60)

# 工作目录
work_dir = "/home/guci/robot_projects/blog/"
Path(work_dir).mkdir(parents=True, exist_ok=True)

# ============================================================
# 1. 创建各个专家Agent（使用不同的LLM）
# ============================================================
print("\n📦 创建专家Agent...")

# General Agent - 通用任务（快速模型）
general_agent = ReactAgentMinimal(
    work_dir=work_dir,
    name="general_agent",
    description="通用任务处理专家 - 处理文件操作等基础任务",
    model="x-ai/grok-code-fast-1",  # grok快速模型
)
print("  ✅ general_agent (grok-code-fast-1)")

# PSM Generation Agent - PSM生成（快速模型）
psm_generation_agent = ReactAgentMinimal(
    work_dir=work_dir,
    name="psm_generation_agent",
    description="PSM生成专家 - 根据PIM生成平台特定模型文档",
    model="x-ai/grok-code-fast-1",  # grok快速模型
    knowledge_files=[
        "knowledge/mda/pim_to_psm_knowledge.md",
        "knowledge/large_file_handling.md"
    ]
)
print("  ✅ psm_generation_agent (grok-code-fast-1)")

# Code Generation Agent - 代码生成（快速模型）
code_generation_agent = ReactAgentMinimal(
    work_dir=work_dir,
    name="code_generation_agent",
    description="代码生成专家 - 根据PSM生成FastAPI代码",
    model="x-ai/grok-code-fast-1",  # grok快速模型
    knowledge_files=[
        "knowledge/mda/generation_knowledge.md"
    ]
)
print("  ✅ code_generation_agent (grok-code-fast-1)")

# Debug Agent - 调试修复（智能模型）
debug_agent = ReactAgentMinimal(
    work_dir=work_dir,
    name="debug_agent",
    description="调试修复专家 - 修复代码和测试问题，需要深度理解",
    model="deepseek-chat",  # DeepSeek智能模型
    knowledge_files=[
        "knowledge/mda/debugging_unified.md"
    ],
    max_rounds=300  # 增加到300轮，确保能完成复杂调试任务
)
print("  ✅ debug_agent (deepseek-chat)")

# ============================================================
# 2. 创建Project Manager Agent
# ============================================================
print("\n👔 创建Project Manager...")

project_manager = ReactAgentMinimal(
    work_dir=work_dir,
    name="project_manager",
    description="项目经理 - 协调其他Agent完成MDA工作流",
    model="x-ai/grok-code-fast-1",  # PM使用快速模型做协调
)
print("  ✅ project_manager (grok-code-fast-1)")

# ============================================================
# 3. 将其他Agent作为Function添加到Project Manager
# ============================================================
print("\n🔗 将专家Agent添加为Project Manager的Function...")

# 使用add_function方法添加Agent
# 每个Agent的execute方法成为可调用的函数
project_manager.add_function(general_agent)
print(f"  ✅ 添加 {general_agent.name}")

project_manager.add_function(psm_generation_agent)
print(f"  ✅ 添加 {psm_generation_agent.name}")

project_manager.add_function(code_generation_agent)
print(f"  ✅ 添加 {code_generation_agent.name}")

project_manager.add_function(debug_agent)
print(f"  ✅ 添加 {debug_agent.name}")

# ============================================================
# 4. Project Manager执行完整工作流
# ============================================================
print("\n" + "=" * 60)
print("🎯 Project Manager执行MDA工作流")
print("=" * 60)

# PIM文件路径
pim_file = "/home/guci/aiProjects/mda/pim-compiler/examples/blog.md"

# Project Manager的任务：用户只需要描述需求，PM自动协调内部Agent
pm_task = f"""
# MDA完整工作流任务

## 需求
从零开始，基于PIM文件生成一个完整的博客系统，包括代码实现和测试。

## 执行步骤
1. **清空工作目录** - 删除所有现有文件，从干净环境开始
2. **生成PSM文档** - 基于PIM生成平台特定模型
3. **生成代码** - 根据PSM生成完整实现
4. **修复测试** - 确保所有测试通过

## 输入
- PIM文件: {pim_file}

## 期望输出
1. PSM文档 (blog_psm.md) - 包含完整的平台特定模型设计
2. 代码实现 - FastAPI应用，包含models、schemas、API endpoints
3. 测试用例 - 单元测试100%通过
4. 项目文档 - README文件

## 验收标准
- 工作目录干净，只包含新生成的文件
- PSM文档完整（包含Domain Models、Service Layer、REST API Design等章节）
- 代码结构清晰（app/目录下有main.py、models.py、schemas.py）
- 测试全部通过（pytest执行无错误）
- 文档齐全（有readme.md说明如何运行）

请从清空目录开始，完成整个MDA工作流。
"""

print("\n📋 用户视角：")
print("  输入: PIM文件 (blog.md)")
print("  输出: 完整的博客系统")
print("\n🎭 Project Manager内部会协调：")
print("  - 文件操作 (grok)")
print("  - PSM生成 (grok)")
print("  - 代码生成 (grok)")
print("  - 测试修复 (deepseek)")
print("\n⚡ LLM自动切换，用户无感知")
print("-" * 60)

# 执行工作流
start_time = time.time()
result = project_manager.execute(task=pm_task)
elapsed = time.time() - start_time

# ============================================================
# 5. 总结
# ============================================================
print("\n" + "=" * 60)
print("✅ MDA工作流完成!")
print("=" * 60)
print(f"\n⏱️ 总耗时: {elapsed:.1f}秒")

if result:
    print(f"\n📊 执行结果摘要:")
    print(result[:500] + "..." if len(result) > 500 else result)

print("\n💡 关键洞察:")
print("  1. ✅ Project Manager是黑盒子 - 用户只描述需求")
print("  2. ✅ 内部Agent自动选择 - PM智能调度")
print("  3. ✅ LLM透明切换 - 用户无需关心")
print("  4. ✅ 通过add_function组合不同LLM的Agent")
print("  5. ✅ 每个Agent独立配置，按需调用")

print("\n🎯 这证明了:")
print("  - Agent作为Function是正确的抽象")
print("  - 组合模式优于内部切换逻辑")
print("  - 用户体验简洁，内部实现灵活")
print("  - 移除内部LLM切换，通过组合实现更优雅")


# 实验

In [2]:
# 测试运行应用
agent = ReactAgentMinimal(
    work_dir="/home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need",
    name="react_agent",
    description="软件系统架构师,专注人工智能agi的实现",
    model="deepseek-chat" #"kimi-k2-turbo-preview" #"x-ai/grok-code-fast-1"  # 使用DeepSeek模型
)


  ✅ 加载知识文件: execution_context_guide.md
  ✅ 加载知识文件: system_prompt_minimal.md
  📚 加载Compact记忆: compact.md
  ✨ 已加载Compact记忆到消息列表
🚀 极简Agent已初始化 [react_agent]
  📍 API: DeepSeek
  🤖 模型: deepseek-chat
  🧠 Compact记忆: 70k tokens触发压缩
  ⚡ Compact记忆替代文件系统
  📚 知识文件: 2个
  ✨ Compact即注意力机制


In [4]:
x = agent.execute(task="""
                  
项目中最近三天被编辑的文件有哪些？（只查看py,ipynb,md文件）
                  
""")
print(x)

  📖 加载语义记忆: /home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need/agent.md

[react_agent] 执行任务...
[react_agent] 📝 任务: 
                  
项目中最近三天被编辑的文件有哪些？（只查看py,ipynb,md文件）
                  
...
[react_agent] ⏰ 时间: 2025-09-19 17:07:37.854856
[react_agent] ============================================================

[react_agent] 🤔 思考第1轮...


[react_agent] 💭 思考: 我来查看项目中最近三天被编辑的py、ipynb、md文件。...

[react_agent] 🔧 调用工具: execute_command
   [react_agent] 📝 command: find . -name "*.py" -o -name "*.ipynb" -o -name "*.md" | xargs ls -lt --time-style=long-iso 2>/dev/n...
   [react_agent] ✅ 结果: -rw-rw-r-- 1 guci guci  36859 2025-09-19 17:07 ./mda_research.ipynb
-rw-rw-r-- 1 guci guci  75395 2025-09-19 17:02 ./notebooks/agent_creator_v2.ipynb


[react_agent] 🤔 思考第2轮...
[react_agent] 💭 思考: 让我查看更详细的最近三天文件修改记录：...

[react_agent] 🔧 调用工具: execute_command
   [react_agent] 📝 command: find . -name "*.py" -o -name "*.ipynb" -o -name "*.md" -exec ls -lt --time-style=long-iso {} + 2>/de...
   [react_agent] ✅ 结果: -rw-rw-r-- 1 guci guci  7557 2025-09-18 05:49 ./knowledge/meta_cognitive_reflex.md
-rw-rw-r-- 1 guci guci 10706 2025-09-18 05:48 ./knowledge/meta_cogn

[react_agent] 🤔 思考第3轮...
[react_agent] 💭 思考: 让我查看更完整的列表，包括今天（9月20日）的文件：...

[react_agent] 🔧 调用工具: execute_command
   [react_agent] 📝 command: find . -name "*.py" -o -name "*.ipynb" -o -na

In [6]:
x = agent.execute(task="""

哪个md文件阐述了知识函数的概念？

""")
print(x)

  📖 加载语义记忆: /home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need/agent.md

[react_agent] 执行任务...
[react_agent] 📝 任务: 

哪个md文件阐述了知识函数的概念？

...
[react_agent] ⏰ 时间: 2025-09-19 17:13:49.431240
[react_agent] ============================================================

[react_agent] 🤔 思考第1轮...
[react_agent] 💭 思考: 让我搜索包含"知识函数"概念的md文件。...

[react_agent] 🔧 调用工具: execute_command
   [react_agent] 📝 command: grep -r "知识函数" --include="*.md" . | head -10
   [react_agent] ✅ 结果: ./knowledge/agent_creator_self_knowledge.md:## 知识函数调用约定
./knowledge/agent_creator_self_knowledge.md:       # 步骤2：提取知识函数interface
./knowledge/agent_cre

[react_agent] 🤔 思考第2轮...
[react_agent] 💭 思考: 让我查看这些文件的具体内容来确认哪个文件详细阐述了知识函数的概念：...

[react_agent] 🔧 调用工具: read_file
   [react_agent] 📝 file_path: knowledge/minimal/system/system_prompt_minimal.md
   [react_agent] 📝 limit: 500
   [react_agent] ✅ 结果: [读取范围: 0-500/8583字符]
# 系统提示词（极简版）

你是一个编程助手。

## 🤖 Agent身份信息

- **我的名字**：{agent_name}
- **我的home目录**：~/.agent/{agent_name}/

In [7]:
x = agent.execute(task="""
你对知识函数的概念如何理解的？
""")
print(x)

  📖 加载语义记忆: /home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need/agent.md

[react_agent] 执行任务...
[react_agent] 📝 任务: 
你对知识函数的概念如何理解的？
...
[react_agent] ⏰ 时间: 2025-09-19 17:16:53.525625
[react_agent] ============================================================

[react_agent] 🤔 思考第1轮...
[react_agent] 💭 思考: 基于对项目语义记忆和代码的分析，我对**知识函数**概念的理解如下：

## 🧠 知识函数的本质理解

### 1. **核心定义**
**知识函数**是在Markdown知识文件中定义的可执行逻辑，通过`@函数名(参数)`格式调用。

### 2. **语法格式**
```markdown
@函数名(参数)
```
- **函数定义**：在知识文件中定义可执行的逻辑步骤
- **函数调用**：直...

[react_agent] ✅ 任务完成（第1轮）
基于对项目语义记忆和代码的分析，我对**知识函数**概念的理解如下：

## 🧠 知识函数的本质理解

### 1. **核心定义**
**知识函数**是在Markdown知识文件中定义的可执行逻辑，通过`@函数名(参数)`格式调用。

### 2. **语法格式**
```markdown
@函数名(参数)
```
- **函数定义**：在知识文件中定义可执行的逻辑步骤
- **函数调用**：直接使用`@函数名(参数)`语法调用
- **参数传递**：支持参数传递，实现动态行为

### 3. **哲学理念：知识即程序**
这是最核心的理念 - **知识文件不仅仅是文档，而是可执行的程序**：
- 📚 知识文件 = 程序源代码
- ⚡ `@函数()` = 函数调用
- 🎯 执行结果 = 程序输出

### 4. **执行机制**
```python
# 在Agent代码中直接调用知识函数
knowledge = @生成初始知识文件(需求描述)
function_interfaces = @提取知

In [9]:
x = agent.execute(task="""

对工作目录中每个文件(只包括py,ipynb,md)，做简要的总结。结果写到files_summary.md中

""")
print(x)

  📖 加载语义记忆: /home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need/agent.md

[react_agent] 执行任务...
[react_agent] 📝 任务: 

对工作目录中每个文件(只包括py,ipynb,md)，做简要的总结。结果写到files_summary.md中

...
[react_agent] ⏰ 时间: 2025-09-19 17:27:49.613802
[react_agent] ============================================================

[react_agent] 🤔 思考第1轮...
[react_agent] 💭 思考: 我来对工作目录中的py、ipynb、md文件进行总结，并生成files_summary.md文件。...

[react_agent] 🔧 调用工具: execute_command
   [react_agent] 📝 command: find . -name "*.py" -o -name "*.ipynb" -o -name "*.md" | grep -v "__pycache__" | grep -v ".git" | so...
   [react_agent] ✅ 结果: ./快速启动指南.md
./问题解决指南.md
./agent_creator.py
./agent.md
./agent_research1.ipynb
./agent_research.ipynb
./agent_research_updated.ipynb
./architecture_int

[react_agent] 🤔 思考第2轮...
[react_agent] 💭 思考: 让我继续查看更多文件：...

[react_agent] 🔧 调用工具: execute_command
   [react_agent] 📝 command: find . -name "*.py" -o -name "*.ipynb" -o -name "*.md" | grep -v "__pycache__" | grep -v ".git" | so...
   [react_agen

In [11]:
x = agent.execute(task="""
/compact            
""")
print(x)

  📖 加载语义记忆: /home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need/agent.md

[react_agent] 执行任务...
[react_agent] 📝 任务: 
/compact            
...
[react_agent] ⏰ 时间: 2025-09-19 17:40:15.450315
[react_agent] ============================================================

🧠 手动触发Compact压缩...
  压缩前: 152 条消息, 约 46116 tokens

🧠 执行Compact压缩...
  ✅ 压缩完成，保留关键信息
  压缩后: 3 条消息, 约 8063 tokens
  压缩率: 82.5%
  💾 已保存到: /home/guci/.agent/react_agent/compact.md
✨ Compact压缩完成！152条消息 → 3条消息


In [ ]:
x = agent.execute(task="""
用自然语言讨论，不要写代码
""")
print(x)

In [ ]:
x = agent.execute(task="""

如何少量修改，使得当前的执行上下文管理支持会话级？

""")

print(x)

In [ ]:
x = agent.execute(task="""

/compact

""")

print(x)

In [ ]:
x = agent.execute(task="""

guci@guci-Latitude-E5470:~/congqing/需求文档/3d_demo (main *%)$ python3 -m http.server 8000
Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
127.0.0.1 - - [08/Sep/2025 22:55:14] "GET /threejs_demo/index.html HTTP/1.1" 304 -
127.0.0.1 - - [08/Sep/2025 22:55:56] "GET /babylonjs_demo/index.html HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 22:55:56] "GET /babylonjs_demo/babylon.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 22:55:56] "GET /babylonjs_demo/babylon.viewer.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 22:55:57] "GET /babylonjs_demo/DamagedHelmet/glTF/DamagedHelmet.gltf HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 22:55:58] "GET /babylonjs_demo/DamagedHelmet/glTF/DamagedHelmet.bin HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 22:55:58] "GET /babylonjs_demo/DamagedHelmet/glTF/Default_normal.jpg HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 22:55:58] code 404, message File not found
127.0.0.1 - - [08/Sep/2025 22:55:58] "GET /babylonjs_demo/DamagedHelmet/glTF/Default_AO.jpg HTTP/1.1" 404 -
127.0.0.1 - - [08/Sep/2025 22:55:58] "GET /babylonjs_demo/DamagedHelmet/glTF/Default_emissive.jpg HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 22:55:58] "GET /babylonjs_demo/DamagedHelmet/glTF/Default_albedo.jpg HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 22:55:58] "GET /babylonjs_demo/DamagedHelmet/glTF/Default_metalRoughness.jpg HTTP/1.1" 200 -


修复问题后，你用 curl --noproxy 验证网页http://localhost:8000/babylonjs_demo/index.html正常
  

""")

print(x)

# 终止程序

In [ ]:
# 停止应用
stop_agent = ReactAgentMinimal(
    work_dir=work_dir,
    name="stop_agent",
    description="应用停止专家",
    model="deepseek-chat"  # 使用DeepSeek模型
)

x = stop_agent.execute(task="""
停止app

# 成功条件
8000端口没有进程

# 知识
当任务完成后，明确说明完成状态并停止执行。
不要重复验证已经成功的操作。
""")

if x is None:
    print("返回结果为空")
else:
    print(f"返回结果：{x[:500]}...")  # 只显示前500个字符

# 验证单元测试

In [ ]:
# 验证单元测试
mda_agent = ReactAgentMinimal(
    work_dir=work_dir,
    name="mda_agent",
    description="MDA验证专家",
    model="deepseek-chat"  # 使用DeepSeek模型
)

x = mda_agent.execute(task="""
# 任务
验证单元测试成功率

# 输出
单元测试成功率
""")

if x is None:
    print("返回结果为空")
else:
    print(f"返回结果：{x[:500]}...")  # 只显示前500个字符

# 分析代码库

In [ ]:
# 分析代码库
analysis_agent = ReactAgentMinimal(
    work_dir="/home/guci/aiProjects/gemini-cli/",
    name="analysis_agent",
    description="代码分析专家",
    model="deepseek-chat"  # 使用DeepSeek模型
)

x = analysis_agent.execute(task="""
# 任务
分析代码库的系统架构和核心组件，以及组件之间的交互
写个文档"gemini分析.md"
""")

if x is None:
    print("返回结果为空")
else:
    print(f"返回结果：{x[:500]}...")  # 只显示前500个字符

In [ ]:
# 分析CLI层执行流程
cli_agent = ReactAgentMinimal(
    work_dir="/home/guci/aiProjects/gemini-cli/",
    name="cli_agent",
    description="CLI分析专家",
    model="deepseek-chat"  # 使用DeepSeek模型
)

x = cli_agent.execute(task="""
CLI 层的执行流程是什么？请用中文回答
""")

if x is None:
    print("返回结果为空")
else:
    print(f"返回结果：{x[:1000]}...")  # 显示前1000个字符